In [27]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from fuzzywuzzy import fuzz

In [2]:
def fetch_job_listings(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()['jobs']
    else:
        print("Error fetching job listings:", response.status_code)
        return None

api_url = "https://zobjobs.com/api/jobs"
json_data = fetch_job_listings(api_url)

In [5]:
def convert_to_dataframe(json_data):
    df = pd.DataFrame(json_data)
    return df

df = convert_to_dataframe(json_data)
df.head()

,created,expires,sourced,unique,companyName,companyUrl,title,description,link,category,location,country
0,2024-03-07T20:23:53+00:00,2024-04-03T20:23,public domain,1406562,Snowflake,https://zobjobs.com/company/false,Senior Frontend Software Engineer,"<div class=""content-intro""><p>Build the future...",https://zobjobs.com/job/senior-frontend-softwa...,[Front End Developer],Bellevue Washington USA,US
1,2024-03-07T20:23:53+00:00,2024-04-03T20:23,public domain,1406561,Social Discovery Group,https://zobjobs.com/company/false,Senior Front-End Developer (React),<div><p><b>(https://socialdiscoverygroup.com/a...,https://zobjobs.com/job/senior-frontend-develo...,[Front End Developer],US,US
2,2024-03-07T20:23:53+00:00,2024-04-03T20:21,public domain,1406560,Hireful,https://zobjobs.com/company/false,Frontend Developer - HTML CSS Javascript PHP Web,Do you have experience developing websites usi...,https://zobjobs.com/job/frontend-developer-htm...,[html],United Kingdom UK,GB
3,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406559,BAE Systems,https://zobjobs.com/company/false,Senior Systems Engineer,Job Description Ready to make a difference? Ou...,https://zobjobs.com/job/senior-systems-enginee...,[systems engineer],United States US,US
4,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406558,Palmer Group,https://zobjobs.com/company/false,CCM Systems Engineer,Our client is looking for a CCM Systems Engine...,https://zobjobs.com/job/ccm-systems-engineer-j...,[systems engineer],United States US,US


In [55]:
import re

def preprocess_data(df):
    """
    Preprocesses the job description column of the DataFrame by removing HTML tags.

    Args:
        df (pandas.DataFrame): DataFrame containing job listings.

    Returns:
        pandas.DataFrame: DataFrame with preprocessed job descriptions.
    """
    # Define regular expression pattern to match HTML tags
    html_tags_pattern = re.compile(r'<[^>]+>')
    # Remove HTML tags from job descriptions
    df['description'] = df['description'].apply(lambda x: re.sub(html_tags_pattern, '', x))
    return df

preprocessed_data = preprocess_data(df)

In [56]:
preprocessed_data

,created,expires,sourced,unique,companyName,companyUrl,title,description,link,category,location,country
0,2024-03-07T20:23:53+00:00,2024-04-03T20:23,public domain,1406562,Snowflake,https://zobjobs.com/company/false,Senior Frontend Software Engineer,Build the future of data. Join the Snowflake t...,https://zobjobs.com/job/senior-frontend-softwa...,[Front End Developer],Bellevue Washington USA,US
1,2024-03-07T20:23:53+00:00,2024-04-03T20:23,public domain,1406561,Social Discovery Group,https://zobjobs.com/company/false,Senior Front-End Developer (React),(https://socialdiscoverygroup.com/about-us/)\n...,https://zobjobs.com/job/senior-frontend-develo...,[Front End Developer],US,US
2,2024-03-07T20:23:53+00:00,2024-04-03T20:21,public domain,1406560,Hireful,https://zobjobs.com/company/false,Frontend Developer - HTML CSS Javascript PHP Web,Do you have experience developing websites usi...,https://zobjobs.com/job/frontend-developer-htm...,[html],United Kingdom UK,GB
3,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406559,BAE Systems,https://zobjobs.com/company/false,Senior Systems Engineer,Job Description Ready to make a difference? Ou...,https://zobjobs.com/job/senior-systems-enginee...,[systems engineer],United States US,US
4,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406558,Palmer Group,https://zobjobs.com/company/false,CCM Systems Engineer,Our client is looking for a CCM Systems Engine...,https://zobjobs.com/job/ccm-systems-engineer-j...,[systems engineer],United States US,US
5,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406557,Abbott Laboratories,https://zobjobs.com/company/false,Systems Engineer II,Job TitleSystems Engineer IIWorking at AbbottA...,https://zobjobs.com/job/systems-engineer-ii-jo...,"[systems engineer, Systems Engineer]",United States US,US
6,2024-03-07T20:23:53+00:00,2024-04-03T20:17,public domain,1406556,BAE Systems Plc,https://zobjobs.com/company/false,Senior Systems Engineer,Ready to make a difference? Our employees des...,https://zobjobs.com/job/senior-systems-enginee...,[systems engineer],Burlington MA MA US,US
7,2024-03-07T20:23:53+00:00,2024-04-03T20:16,public domain,1406555,BAE Systems,https://zobjobs.com/company/false,Senior Model Based Systems Engineer,Job Description\n\nSee what you’re missing. Ou...,https://zobjobs.com/job/senior-model-based-sys...,[systems engineer],Burlington Massachusetts US,US
8,2024-03-07T20:23:53+00:00,2024-04-03T20:15,public domain,1406554,BAE Systems,https://zobjobs.com/company/false,Senior Systems Engineer,Job Description Ready to make a difference? Ou...,https://zobjobs.com/job/senior-systems-enginee...,[systems engineer],Burlington Massachusetts United States US,US
9,2024-03-07T20:23:53+00:00,2024-04-03T20:14,public domain,1406553,Abbott,https://zobjobs.com/company/false,Systems Engineer II,Abbott is a global healthcare leader that help...,https://zobjobs.com/job/systems-engineer-ii-jo...,"[systems engineer, Systems Engineer]",Sylmar California US,US


In [10]:
# Step 3: Feature Engineering
def extract_features(df):
    # Example: Use TF-IDF to convert job descriptions into numerical vectors
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'])
    return tfidf_matrix

features = extract_features(preprocessed_data)

In [11]:
features

<20x1755 sparse matrix of type '<class 'numpy.float64'>'
	with 5625 stored elements in Compressed Sparse Row format>

In [12]:
# Step 4: Build Recommender Model
def build_recommender_model(features):
    # Example: Use cosine similarity for content-based recommendation
    cosine_sim = linear_kernel(features, features)
    return cosine_sim

model = build_recommender_model(features)

In [47]:
def get_recommendations(job_title, df, model, top_n=5):
    if len(df) == 0:
        print("DataFrame is empty. Unable to make recommendations.")
        return None
    # Calculate similarity scores between the input job title and all job titles in the DataFrame
    similarity_scores = df['title'].apply(lambda x: fuzz.token_sort_ratio(job_title.lower(), x.lower()))
    # Get the index of the job title with the highest similarity score
    idx = similarity_scores.idxmax()
    # Use the index to get recommendations based on the model
    sim_scores = list(enumerate(model[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    job_indices = [i[0] for i in sim_scores]
    return df.iloc[job_indices]['title']

In [51]:
def get_recommendations(job_title, df, model, top_n=5):
    """
    Generates recommendations based on a given job title.

    Args:
        job_title (str): The title of the job to generate recommendations for.
        df (pandas.DataFrame): DataFrame containing job listings.
        model (numpy.ndarray): Cosine similarity matrix of job descriptions.
        top_n (int): Number of recommendations to generate.

    Returns:
        list: List of dictionaries containing recommended job details.
    """
    if len(df) == 0:
        print("DataFrame is empty. Unable to make recommendations.")
        return None
    # Calculate similarity scores between the input job title and all job titles in the DataFrame
    similarity_scores = df['title'].apply(lambda x: fuzz.token_sort_ratio(job_title.lower(), x.lower()))
    # Get the index of the job title with the highest similarity score
    idx = similarity_scores.idxmax()
    # Use the index to get recommendations based on the model
    sim_scores = list(enumerate(model[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    job_indices = [i[0] for i in sim_scores]
    # Format recommendations in the specified format
    recommendations = []
    for index in job_indices:
        job_id = df.loc[index, 'unique']
        title = df.loc[index, 'title']
        description = df.loc[index, 'description']
        recommendations.append({
            'job_id': job_id,
            'title': title,
            'description': description
        })
    return recommendations

In [52]:
import pickle
def load_pickled_model(filename):
    """
    Deserializes the model from a pickled file.

    Args:
        filename (str): The name of the pickled file containing the model.

    Returns:
        object: The deserialized model object.
    """
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    return model

In [53]:
model_filename = "./recommender_model.pkl"
model = load_pickled_model(model_filename)

In [54]:
# Example usage
job_title = "systems"
recommended_jobs = get_recommendations(job_title, preprocessed_data, model)
print("Recommended Jobs for {}: \n{}".format(job_title, recommended_jobs))

Recommended Jobs for systems: 
[{'job_id': 1406558, 'title': 'CCM Systems Engineer', 'description': 'Our client is looking for a CCM Systems Engineer with experience managing CCM solutions using SmartCOMM. In this role, you will be responsible for establishing requirements from the business and implementing this solution. This is a great opportunity to work for a company that offers a fun, challenging, and work-life balance environment. Apply today if interested!<br><br>Responsibilities<br>• Partner with key stakeholders to define requirements for the CCM system<br>• Responsible for communicating to the business and technical teams with any updates related to SmartCOMM<br>• Manage System availability and SLA reporting<br>• Oversee Scheduling and ad-hock job requests<br>• Monitor and trend form usage to establish an audit and retention practice<br>• Work with technical teams to set integrations between CCM and other systems<br>• Maintain SmartCOMM documentation, engagement rules, and st